# Inheritance : for good or for worse

This chapter is about inheritance and subclassing, with emphasis on two particulars that are very specific to Python 

- The pitfalls of subclassing from built-in types
- Multiple inheritance and the method resolution order.

The issue of subclassing built-ins. The rest of the chapter will cover multiple inheritance with our case studies and discuss good and bad practice when buliding class hierachies.

## 1. Subclassing built-in types is tricky

The code of the bulit-ins(list or dict) does not call special mehtods overriden by user-defined classes.

Officially, CPython has no rule at all for when exactly overriden method of subclass of built-in types get implicitly called or not.

For example, an overrideen `__getitem()__` in a subclass of `dict` will not be called by the built-in `get()` method.

In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

In [2]:
# The __init__ method inherited from dict clearly 
# ignored that __settime__ was overridden. 
# the value of 'one' is not duplicated
dd = DoppelDict(one=1)
dd

{'one': 1}

In [3]:
# The [] operator calls out __setitem__ and works as expected : 
# 'two' maps to the duplicated value [2,2]
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [4]:
# The update method from dict does not use our version of __setitem__ either:
# the value of 'three' was not duplicated
dd.update(three=3)
dd

{'one': 1, 'three': 3, 'two': [2, 2]}

This built-in behavior is a violation of a basic rule of oop

the `__missing__` method works as documented only because it's handled as a special case.

also happens with overriden methods of other classes that should be called by the built-in methods.

In [5]:
class AnswerDict(dict):
    # __getitem__ always returns 42, no matter what the key.
    def __getitem__(self, key):
        return 42

In [6]:
# ad is an AnswerDict loaded with key-value pair('a', 'foo')
ad = AnswerDict(a='foo')
ad['a']

42

In [7]:
# d is an instance of plain dict, which we update with ad.
d = {}
# The dict.update method ignored out AnswerDict.__getitem__.
d.update(ad)
d['a']

'foo'

In [8]:
d

{'a': 'foo'}

Subclassing built-in type like `dict` or `list` or `str` directily is error-prone because the built-in methods mostly ignore user-defined overrides.

Instead of subclassing the built-ins, derive your classes from `UserDict`, `UserList`and `UserString` from `collection` module.

In [1]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)
        

In [10]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [11]:
dd['two'] = 2
dd

{'two': [2, 2], 'one': [1, 1]}

In [12]:
dd.update(three=3)
dd

{'two': [2, 2], 'three': [3, 3], 'one': [1, 1]}

In [2]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42
    

In [3]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [4]:
d = {}
d.update(ad)
d['a']

42

In [5]:
d

{'a': 42}

To summarize : the problem described in this section applies only to method delegation within the C language implementation of the built-in types and only affects user-defined classes derived directly from those types.

If you subclass from a class coded in Python, such as `UserDict` or `MutableMapping`, you will not be troubled by this.

## 2. Multiple Inheritance and method resolution

How does Python decide which attribute to use if superclasses from parallel branches define attributes with the same name?

Any language implementing multiple inheritance need to deal with potential naming conflicts when unrelated ancestor classes implement a method by the same name.

In [6]:
class A:
    def ping(self):
        print('ping_A:', self)
        
class B(A):
    def pong(self):
        print('pong_B', self)
        
class C(A):
    def pong(self):
        print('PONG_C:', self)
        
class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping_D:', self)
        
    def pingpong(self):
        self.ping()
        # super.ping() which bypasses the ping in D and finds the ping method in A
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [7]:
d = D()
# Simply calling d.pong() cause the B version to run.
d.pong()

pong_B <__main__.D object at 0x00000222E5ECF128>


In [8]:
# You can always call a method on a superclass directly, passing the instance 
# as an explicity argument.
C.pong(d)

PONG_C: <__main__.D object at 0x00000222E5ECF128>


The ambiguity of a call like `d.pong()` is resolved because Python follows a specific order when traversing the inheritance graph. That order is called `MRO`: Method Resolution Order.

Classes have an attribute called `__mro__` holding a tuple of references to the superclasses in MRO oder, from the current class all the way to the `object` class.

In [10]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

The recommended way to delegate method calls to superclasses if the `super()` built-on function


However, it's also possible, and sometimes convenient, to bypass the MRO and invoke a method on a superclass directly.
```python
def ping(self):
    A.ping(self) #instead of super().ping()
    print('post-ping', self)
```

Note that when calling an instance method directly on a class, you must pass `self` explicitly, because you are accessing an `unbound method`

In [9]:
# Using super() to call ping
d = D()
# the first call is super().ping() ; the super delegates the ping call to class A
d.ping()

ping_A: <__main__.D object at 0x00000222E5ECFDA0>
post-ping_D: <__main__.D object at 0x00000222E5ECFDA0>


In [10]:
d = D()
d.pingpong()

ping_A: <__main__.D object at 0x00000222E5ECFE48>
post-ping_D: <__main__.D object at 0x00000222E5ECFE48>
ping_A: <__main__.D object at 0x00000222E5ECFE48>
pong_B <__main__.D object at 0x00000222E5ECFE48>
pong_B <__main__.D object at 0x00000222E5ECFE48>
PONG_C: <__main__.D object at 0x00000222E5ECFE48>


```python
# self.ping() runs the ping method of D, which outputs this line and the next one.
ping: <__main__.D object at 0x000002B96BF74BA8>

post-ping: <__main__.D object at 0x000002B96BF74BA8>
# super.ping() which bypasses the ping in D and finds the ping method in A.
ping: <__main__.D object at 0x000002B96BF74BA8>
# self.pong() which finds the B implementation of pong, according to the __mro__
pong <__main__.D object at 0x000002B96BF74BA8>
# super.pong() which finds the same B.pong implementation, also following the __mro__
pong <__main__.D object at 0x000002B96BF74BA8>
# C.pong(self) which finds the C.pong implementation, ignoring the __mro__.
PONG: <__main__.D object at 0x000002B96BF74BA8>
    
```

The MRO taks into account not only the inheritance graph but also the order in which superclasses are listed in a subclass declaration.

In other words, the D class was declared as class `D(C,B) :`, the `__mro__` of class D would be different : C would be searched before `B`.

In [13]:
bool.__mro__

(bool, int, object)

In [14]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [15]:
print_mro(bool)

bool, int, object


In [17]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self): 
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    def __len__(self): 
        return len(self._cards)
    def __getitem__(self, position): 
        return self._cards[position]
    def __setitem__(self, position, value): # 
        self._cards[position] = value
    def __delitem__(self, position): # 
        del self._cards[position]
    def insert(self, position, value): # 
        self._cards.insert(position, value)

In [18]:
print_mro(FrenchDeck2)

FrenchDeck2, MutableSequence, Sequence, Sized, Iterable, Container, object


In [19]:
import numbers
print_mro(numbers.Integral)

Integral, Rational, Real, Complex, Number, object


In [20]:
import io
print_mro(io.BytesIO)

BytesIO, _BufferedIOBase, _IOBase, object


In [21]:
print_mro(io.TextIOWrapper)

TextIOWrapper, _TextIOBase, _IOBase, object


The MRO is computed using an algorithm called `C3`

In [23]:
import tkinter

print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


## 3. Mutiple Inheritance in the real world

pros and cons of multiple inheritance

Good use : Adapter pattern, `collections.abc` package, Tkinter GUI toolkit.

## 4. Coping with multiple inheritance

Inheritance is used for different reasons, and multiple inheritance adda alternatives and complexity.

### 1. Distinguish interface inheritance from implementation inheritance.

Keep straight the reasons why subclassing is done in the first place.

- Inheritance of interface : creates a sub-type, imliying an "is-a" relationship.
- Inheritance of implementation : avoids code duplication by reuse.

In practice both uses are often simultaneous, but whenever you can make the intent clear, do it.

Inheritance for code reuse is an implementation detail, and it can often be replaced by composition and delegation.

Inheritance vs interface

https://stackoverflow.com/questions/5816563/when-should-i-choose-inheritance-over-an-interface-when-designing-c-sharp-class/5816589


### 2. Make interfaces explicity with ABCs

In modern Python, if a class is desinged to define an interface, it should be an explict ABC.

**subclass `abc.ABC` or another ABC**

### 3. Use mixins for code reuse

If a class is designed to provide method implementations for reuse by multipe unrelated subclasses, without implying an "is-a" relationship, it shoud be an explict `mixin` class.

A mixin should never be instantiated, and concrete classes should not inherit only from ax mixin.

Each mixin should provide a single specific behavior, implementing few and very closely related methods.

A mixin is a special kind of multiple inheritance. There are two main situdations where mixins are used: 
    1. You want to provide a lot of optional features for a class
    2. You want to use one particular feature in a lot of different classes.
    
클래스에 추가적인 속성이나 메소드를 제공

http://hamait.tistory.com/859

https://stackoverflow.com/questions/533631/what-is-a-mixin-and-why-are-they-useful

### 4. Make mixins explicity by naming

There is no formal way in Python to state that a class is a mixin, so it is highly recommended that they are named with a `...Mixin` suffix. 

### 5. An ABC may also be a mixin ; the reverse is not true

since an ABC can implement concrete methods, it works as a mixin as well.

An ABC also defines a type, which a mixin does not. 

An ABC can be the sole base class of any another class, while a mixin should never be subclassed alone except by another. more specialized mixin.

One restriction applies to ABCs and not to mixins: the concrete mehtods implemented in an ABC should only collaborate with methods of the same ABC and its superclasses.



### 6. Don't subclass from more than one concrete class

Concrete classes should have zero or at most one concrete superclass. 

all but one of the superclasses of a concrete class should be ABCs or mixins.

```python
class MyConcreteClass(Alpha, Beta, Gamma):
    """This is a concrete : it can be instantiated"""
```

if `Alpha` is a concrete class, then `Beta` and `Gamma` must be ABCs or mixins

### 7. Provide aggregate classes to users

If some combination of ABCs or mixins in particularly useful to clientt code, provide a class that brings them together in a sensible way. -> `aggregate class`

"A class that is constructed primarily by inheriting from mixins and does not add its own structure or behavior is called an `aggregate class`."

```python
class Widget(BaseWidget, Pack, Place, Grid):
    """
    Internal class.
    
    Base class for a widget which can be positioned 
    with the geometry managers Pack, Place or Grid.
    """
    pass
```

The body of `Widget` is empty, but the class provides a useful service : it brings together four superclasses so that anyone who needs to create a new widget does not need remeber all those mixins, or wonder if they need to be declared in a certain order in a `class` statement.


### 8. "Favor object composition over class inheritance"

Favoring composition leads to more flexible designs. 

widget instances couldhold a reference to a geometry manager, and invoke its methods.

After all, a `Widget` should not "be" a geometry manager, but could use the services of one via delegation. Then you could add a new geometry manager without touching the widget class hieracrchy and without worrying about name clashes.

Composition and delegation can replace the use of mixins to make behaviors avaiable to different classes, but cannot replace the use of interface inheritance to define a hierachy of types.

## 5. modern example: mixins in Django 

The original generic views were functions, so they were not extensible.

The base classes and mixins are in the `base` module of the `django.views.generic` package.

Concrete subclasses of `View` are supposed to implement the handler metheods, so why aren't they part of the `View` interface?

The reason : subclasses are free to implement just the handlers they want to support.

If an `HTTP POST` request is sent to a `TemplateVeiw`, the inherited `View.dispatch` method checks that there is no `post` handler, and  produces an response.



## 6. Chapter summary

The problem with subclassing bulit-in types : their native methods implemented in C do not call overridden methods in subclasses, except in very few special cases.

It's easier to subclass `UserList`, `UserDict` or `UserString` which actually wrap the built-in types and delegate operation to them 

If the desired behavior is very different from what the built-ins offer, it may be easier to subclass the appropriate ABC from `collections.abc` and write your own implmentation.



- Mehtod resolution order, encoded in the `__mro__` class attribute, addresses the problem of potential naming conflicts in inherited methods.

- How the `super()` built-in follows the `__mro__` to call a method on a superclass.

In [1]:
import abc

class Bird(abc.ABC):
    @abc.abstractmethod
    def fly(self):
        pass
    
class Parrot(Bird):
    def fly(self):
        print("Flying")


p = Parrot()


In [2]:
class Aeroplane(abc.ABC):
    @abc.abstractmethod
    def fly(self):
        pass


class Boeing(Aeroplane):
    def fly(self):
        print("Flying!")

b = Boeing()

In [3]:
isinstance(p, Aeroplane)

False

In [4]:
def test(entity):
    entity.fly()
    
test(p)

Flying


In [5]:
test(b)

Flying!
